In [1]:
import pickle
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows', None)
pd.options.display.max_colwidth = 200

data = None
regex_decimal = r'(\d+)'
regex_float = r'([-+]?\d*\.*\d+)'
regex_float_with_dec_part = r'(\d+\.\d+)'

# Raw data, source: mediamarkt.es, web scraping 
# Working on the backup version from 19.03.2020 to provide that the code interprets without errors. 
# The set is updated frequently and adjustments may be needed when used with new phones.
with open('data_raw_backup.pickle', 'rb') as handle:
    smartphones = pickle.load(handle) # list of dictionaries, each dictionary describes one smartphone
    data = pd.DataFrame.from_dict(smartphones)

In [2]:
# Matching columns to slot names in Clips 'smartphone' template
columns = {'Marca' : "brand",
          'Tipo:' : "model",
          'Sistema operativo:' : "operating_system",
          'Velocidad Procesador:' : "cpu",
          'Capacidad memoria:' : "memory",
          'Tamaño memoria RAM:' : "ram",
          'Resolución cámara trasera:' : "camera",
          'Peso:' : "weight",
          'Tamaño pantalla (cm):' : "screen_diagonal",
          'Capacidad batería:' : "battery",
          'Precio' : "price",
          'Admite SD:' : "sd",
          'GPS:' : "gps",
          'Bluetooth:' : "bluetooth",
          'Wi-Fi:' : "wifi",
          'Link' : "link"
          }

In [3]:
# Filtering the columns to eliminate unused ones and renaming
data = data[list(columns)]
data = data.rename(columns=columns)
data

,brand,model,operating_system,cpu,memory,ram,camera,weight,screen_diagonal,battery,price,sd,gps,bluetooth,wifi,link
0,huawei,P30 Lite,Android,2.2 GHz,128 GB,4 GB,48 megapixel,159 g,15.62 cm,3340 mAh,"229,-",Sí,Sí,Sí,Sí,/es/product/_m%C3%B3vil-huawei-p30-lite-azul-128-gb-4-gb-ram-6-15-full-hd-kirin-710-3340-mah-android-1452688.html
1,xiaomi,Redmi Note 8 Pro,Android,"2,05 GHz",128 GB,6 GB,64 pixels,200 g,NaN,4500 mAh,"228,-",Sí,Sí,Sí,Sí,/es/product/_m%C3%B3vil-xiaomi-redmi-note-8-pro-azul-128-gb-6-gb-ram-6-53-full-hd-helio-g90t-4500-mah-android-1469588.html
2,xiaomi,Redmi Note 8T,Android,2.0 GHz,64 GB,4 GB,NaN,190 g,16 cm,4000 mAh,"199,-",Sí,Sí,Sí,Sí,/es/product/_m%C3%B3vil-xiaomi-redmi-note-8t-azul-64-gb-4-gb-ram-6-3-full-hd-snapdragon-665-4000-mah-android-1468593.html
3,xiaomi,Mi 9,Android,2.84 GHz,128 GB,6 GB,48 megapixel,173 g,16.25 cm,3300 mAh,"369,-",no,Sí,Sí,Sí,/es/product/_m%C3%B3vil-xiaomi-mi-9-azul-128-gb-6-gb-ram-6-4-full-hd-snapdragon-855-3300-mah-android-1447026.html
4,xiaomi,Redmi Note 8T,Android,2.0 GHz,64 GB,4 GB,48 megapixel,190 g,16 cm,4000 mAh,"199,-",Sí,Sí,Sí,Sí,/es/product/_m%C3%B3vil-xiaomi-redmi-note-8t-gris-64-gb-4-gb-ram-6-3-full-hd-snapdragon-665-4000-mah-android-1468591.html
5,samsung,Galaxy A40,Android,1.8 GHz,64 GB,4 GB,16 megapixel,162 g,14.98 cm,3100 mAh,"199,-",Sí,Sí,Sí,Sí,/es/product/_m%C3%B3vil-samsung-galaxy-a40-azul-64-gb-4-gb-ram-5-9-samsung-exynos-3100-mah-android-1446603.html
6,huawei,P Smart (2019),Android,1.7 GHz,64 GB,3 GB,NaN,160 g,15.77 cm,3400 mAh,"159,-",Sí,Sí,Sí,Sí,/es/product/_m%C3%B3vil-huawei-p-smart-2019-azul-64-gb-3-gb-ram-6-21-full-hd-kirin-710-3400-mah-android-1442814.html
7,samsung,Galaxy S10,Android,2.7 GHz,128 GB,8 GB,NaN,NaN,15.49 cm,3400 mAh,"649,-",Sí,Sí,Sí,Sí,/es/product/_m%C3%B3vil-samsung-galaxy-s10-negro-128-gb-8-gb-ram-6-1-qhd-exynos-9-9821-3400-mah-android-1446423.html
8,huawei,Y5 (2019),Android,2.0 GHz,16 GB,2 GB,13 megapixel,NaN,14.47 cm,3020 mAh,"99,-",Sí,Sí,Sí,Sí,/es/product/_m%C3%B3vil-huawei-y5-2019-negro-16-gb-2-gb-ram-5-71-hd-mt6761-3020-mah-android-1459740.html
9,samsung,Galaxy A40,Android,1.8 GHz,64 GB,4 GB,16 megapixel,162 g,14.98 cm,3100 mAh,"199,-",Sí,Sí,Sí,Sí,/es/product/_m%C3%B3vil-samsung-galaxy-a40-coral-64-gb-4-gb-ram-5-9-samsung-exynos-3100-mah-android-1446604.html


In [4]:
# Brand
# Hand-check if values overlap due to unconsistent spelling
data.brand.value_counts()

apple        75
samsung      54
huawei       31
lg           21
honor        19
sony         19
xiaomi       17
motorola     14
hammer       10
oppo          9
alcatel       9
nokia         6
crosscall     3
tcl           2
meizu         2
realme        2
vsmart        1
gigaset       1
palm          1
moto          1
weimei        1
lenovo        1
oneplus       1
asus          1
Name: brand, dtype: int64

In [5]:
# Operating system
# replace "Android 6.0" by "Android"
data.loc[data.operating_system == "Android 6.0", "operating_system"] = "Android"
# replace "Propietario" by "other"
data.loc[data.operating_system == "Propietario", "operating_system"] = "other"
data.operating_system.value_counts()

Android    200
iOS         74
other        1
Name: operating_system, dtype: int64

In [6]:
# CPU
# Identified units: GHz|MHz|nm. 
# One value doesn't have unit -> hand edit
data.loc[data.cpu == "1.3", "cpu"] = "1.3 GHz"
# assert that all values have identified units
assert data.cpu.str.contains(pat='GHz|MHz|nm', na=True).all()
data.cpu = data.cpu.str.replace(',', '.')
# mark MHz and nm
data['cpu_MHz'] = data.cpu.str.contains(pat='MHz', na=False)
data['cpu_nm'] = data.cpu.str.contains(pat='nm', na=False)
# mark multiple cores cpu
data['cpu_mult'] = data.cpu.str.contains(pat='x|-', na=False)
# extract largest value from multiple cores cpu
data.cpu = data.apply(lambda x: max(re.findall(regex_float_with_dec_part,x.cpu)) if x.cpu_mult is True else x.cpu, axis=1)
# change values to float
data.cpu = data.cpu.str.extract(regex_float)
data.cpu = data.apply(lambda x: pd.to_numeric(x.cpu), axis=1)
# change MHz to GHz
data.cpu = data.apply(lambda x: x.cpu/1000 if x.cpu_MHz is True else x.cpu, axis=1)
# change nm to negative number, eg. -7
data.cpu = data.apply(lambda x: -x.cpu if x.cpu_nm is True else x.cpu, axis=1)
data.pop('cpu_MHz')
data.pop('cpu_nm')
data.pop('cpu_mult')
data.shape

(301, 16)

In [7]:
# Memory
# Most values GB, some MB exist
# assert that all values are either MB or GB
assert data.memory.str.contains(pat='MB|GB', na=True).all()

data['memory_mb'] = data.memory.str.contains(pat='MB', na=False)
data.memory = data.memory.str.extract(regex_decimal)
data.memory = pd.to_numeric(data.memory)
# for each row containing MB calculate to GB
data.memory = data.apply(lambda x: x.memory / 1024 if x.memory_mb is True else x.memory, axis=1)
data.pop('memory_mb')
data.shape

(301, 16)

In [8]:
# RAM
# similar as memory treatment
assert data.ram.str.contains(pat='MB|GB', na=True).all()

data['ram_mb'] = data.ram.str.contains(pat='MB', na=False)
data.ram = data.ram.str.extract(regex_decimal)
data.ram = pd.to_numeric(data.ram)
data.ram = data.apply(lambda x: x.ram / 1024 if x.ram_mb is True else x.ram, axis=1)
data.pop('ram_mb')
data.shape

(301, 16)

In [9]:
# Camera
# Identified units: megapixel, pixels (refers to megapixels)
# Assert that all entries have identified units
assert data.camera.str.contains(pat='megapixel|pixels', na=True).all()
data.camera = data.camera.str.extract(regex_decimal)
data.shape

(301, 16)

In [10]:
# Weight
# Identified units: g, gr, kg (=1000g)
# Assert that all entries have identified units
assert data.weight.str.contains(pat='g|gr|kg', na=True).all()
data['weight_kg'] = data.weight.str.contains(pat='kg', na=False)
data.weight = data.weight.str.extract(regex_float)
data.weight = pd.to_numeric(data.weight)
# for each row containing kg calculate to g
data.weight = data.apply(lambda x: x.weight * 1000 if x.weight_kg is True else x.weight, axis=1)
data.pop('weight_kg')
# assert that all values (drop NaN) are within reasonable range
assert data.weight.dropna().between(100,300).all()
data.shape

(301, 16)

In [11]:
# Screen diagonal
# Identified units: cm
# Assert that all entries have identified units
assert data.screen_diagonal.str.contains(pat='cm', na=True).all()
data.screen_diagonal = data.screen_diagonal.str.extract(regex_float)
data.screen_diagonal = pd.to_numeric(data.screen_diagonal)
# assert that all values (drop NaN) are within reasonable range
assert data.screen_diagonal.dropna().between(5,21).all()
data.shape

(301, 16)

In [12]:
# Battery
# Identified units: mAh
assert data.battery.str.contains(pat='mAh', na=True).all()
data.battery = data.battery.str.extract(regex_decimal)
data.battery = pd.to_numeric(data.battery)
# assert that all values (drop NaN) are within reasonable range
assert data.battery.dropna().between(0,8000).all()
data.shape

(301, 16)

In [13]:
# Price
data.price = data.price.str.replace(',', '.')
data.price = data.price.str.extract(regex_decimal)
data.price = pd.to_numeric(data.price)
# assert that all values (drop NaN) are within reasonable range AND that all values are given
assert data.price.dropna().between(0,4000).all()
assert not(data.price.isna().all())
data.shape

(301, 16)

In [14]:
# Binary columns: sd gps bluetooth wifi
binary_columns = [data.sd, data.gps, data.bluetooth, data.wifi]
pd.concat(binary_columns).value_counts()
# Identified values: Sí, no
def fun(x): 
 return x.str.replace('Sí', 'y').replace('no', 'n')
data.sd = fun(data.sd)
data.gps = fun(data.gps)
data.bluetooth = fun(data.bluetooth)
data.wifi = fun(data.wifi)
data.shape

(301, 16)

In [15]:
data

,brand,model,operating_system,cpu,memory,ram,camera,weight,screen_diagonal,battery,price,sd,gps,bluetooth,wifi,link
0,huawei,P30 Lite,Android,2.200,128.00000,4.0,48,159.0,15.62,3340.0,229,y,y,y,y,/es/product/_m%C3%B3vil-huawei-p30-lite-azul-128-gb-4-gb-ram-6-15-full-hd-kirin-710-3340-mah-android-1452688.html
1,xiaomi,Redmi Note 8 Pro,Android,2.050,128.00000,6.0,64,200.0,NaN,4500.0,228,y,y,y,y,/es/product/_m%C3%B3vil-xiaomi-redmi-note-8-pro-azul-128-gb-6-gb-ram-6-53-full-hd-helio-g90t-4500-mah-android-1469588.html
2,xiaomi,Redmi Note 8T,Android,2.000,64.00000,4.0,NaN,190.0,16.00,4000.0,199,y,y,y,y,/es/product/_m%C3%B3vil-xiaomi-redmi-note-8t-azul-64-gb-4-gb-ram-6-3-full-hd-snapdragon-665-4000-mah-android-1468593.html
3,xiaomi,Mi 9,Android,2.840,128.00000,6.0,48,173.0,16.25,3300.0,369,n,y,y,y,/es/product/_m%C3%B3vil-xiaomi-mi-9-azul-128-gb-6-gb-ram-6-4-full-hd-snapdragon-855-3300-mah-android-1447026.html
4,xiaomi,Redmi Note 8T,Android,2.000,64.00000,4.0,48,190.0,16.00,4000.0,199,y,y,y,y,/es/product/_m%C3%B3vil-xiaomi-redmi-note-8t-gris-64-gb-4-gb-ram-6-3-full-hd-snapdragon-665-4000-mah-android-1468591.html
5,samsung,Galaxy A40,Android,1.800,64.00000,4.0,16,162.0,14.98,3100.0,199,y,y,y,y,/es/product/_m%C3%B3vil-samsung-galaxy-a40-azul-64-gb-4-gb-ram-5-9-samsung-exynos-3100-mah-android-1446603.html
6,huawei,P Smart (2019),Android,1.700,64.00000,3.0,NaN,160.0,15.77,3400.0,159,y,y,y,y,/es/product/_m%C3%B3vil-huawei-p-smart-2019-azul-64-gb-3-gb-ram-6-21-full-hd-kirin-710-3400-mah-android-1442814.html
7,samsung,Galaxy S10,Android,2.700,128.00000,8.0,NaN,NaN,15.49,3400.0,649,y,y,y,y,/es/product/_m%C3%B3vil-samsung-galaxy-s10-negro-128-gb-8-gb-ram-6-1-qhd-exynos-9-9821-3400-mah-android-1446423.html
8,huawei,Y5 (2019),Android,2.000,16.00000,2.0,13,NaN,14.47,3020.0,99,y,y,y,y,/es/product/_m%C3%B3vil-huawei-y5-2019-negro-16-gb-2-gb-ram-5-71-hd-mt6761-3020-mah-android-1459740.html
9,samsung,Galaxy A40,Android,1.800,64.00000,4.0,16,162.0,14.98,3100.0,199,y,y,y,y,/es/product/_m%C3%B3vil-samsung-galaxy-a40-coral-64-gb-4-gb-ram-5-9-samsung-exynos-3100-mah-android-1446604.html


In [16]:
with open('data_clean.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)